# Title: Personalized Medical Recommendation System with Machine Learning

# Description:

Welcome to our cutting-edge Personalized Medical Recommendation System, a powerful platform designed to assist users in understanding and managing their health. Leveraging the capabilities of machine learning, our system analyzes user-input symptoms to predict potential diseases accurately.

# load dataset & tools

In [1]:
import  pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [2]:
dataset = pd.read_csv('./datasets/Training.csv')

In [3]:
dataset

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,(vertigo) Paroymsal Positional Vertigo
4916,0,1,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,Acne
4917,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Urinary tract infection
4918,0,1,0,0,0,0,1,0,0,0,...,0,0,1,1,1,1,0,0,0,Psoriasis


In [4]:
dataset.shape

(4920, 133)

In [24]:
keywords_new = {
    "itching": ["itching", "itch"],
    "skin_rash": ["skin rash", "rash"],
    "nodal_skin_eruptions": ["nodal skin eruptions"],
    "continuous_sneezing": ["continuous sneezing"],
    "shivering": ["shivering", "chills"],
    "chills": ["chills", "fever"],
    "joint_pain": ["joint pain", "arthritis"],
    "stomach_pain": ["stomach pain", "abdominal pain"],
    "acidity": ["acidity", "heartburn"],
    "ulcers_on_tongue": ["ulcers on tongue", "mouth sores"],
    "muscle_wasting": ["muscle wasting", "muscle weakness"],
    "vomiting": ["vomiting", "nausea"],
    "burning_micturition": ["burning micturition", "painful urination"],
    "spotting_urination": ["spotting urination", "frequent urination"],
    "fatigue": ["fatigue", "tiredness"],
    "weight_gain": ["weight gain", "obesity"],
    "anxiety": ["anxiety", "stress"],
    "cold_hands_and_feets": ["cold hands and feet", "poor circulation"],
    "mood_swings": ["mood swings", "irritability"],
    "weight_loss": ["weight loss", "slimming"],
    "restlessness": ["restlessness", "insomnia"],
    "lethargy": ["lethargy", "apathy"],
    "patches_in_throat": ["patches in throat", "sore throat"],
    "irregular_sugar_level": ["irregular sugar level", "diabetes"],
    "cough": ["cough", "respiratory issues"],
    "high_fever": ["high fever", "temperature elevation"],
    "sunken_eyes": ["sunken eyes", "fatigue"],
    "breathlessness": ["breathlessness", "shortness of breath"],
    "sweating": ["sweating", "perspiration"],
    "dehydration": ["dehydration", "water loss"],
    "indigestion": ["indigestion", "heartburn"],
    "headache": ["headache", "migraine"],
    "yellowish_skin": ["yellowish skin", "jaundice"],
    "dark_urine": ["dark urine", "kidney issues"],
    "nausea": ["nausea", "vomiting"],
    "loss_of_appetite": ["loss of appetite", "decreased hunger"],
    "pain_behind_the_eyes": ["pain behind the eyes", "eye strain"],
    "back_pain": ["back pain", "muscle strain"],
    "constipation": ["constipation", "bowel issues"],
    "abdominal_pain": ["abdominal pain", "stomach cramps"],
    "diarrhoea": ["diarrhoea", "frequent bowel movements"],
    "mild_fever": ["mild fever", "low-grade temperature"],
    "yellow_urine": ["yellow urine", "urine color change"],
    "yellowing_of_eyes": ["yellowing of eyes", "jaundice"],
    "acute_liver_failure": ["acute liver failure", "liver dysfunction"],
    "fluid_overload": ["fluid overload", "edema"],
    "swelling_of_stomach": ["swelling of stomach", "abdominal distension"],
    "swelled_lymph_nodes": ["swelled lymph nodes", "enlarged lymph nodes"],
    "malaise": ["malaise", "general discomfort"],
    "blurred_and_distorted_vision": ["blurred and distorted vision", "eye problems"],
    "phlegm": ["phlegm", "mucus"],
    "throat_irritation": ["throat irritation", "sore throat"],
    "redness_of_eyes": ["redness of eyes", "eye irritation"],
    "sinus_pressure": ["sinus pressure", "congestion"],
    "runny_nose": ["runny nose", "rhinorrhea"],
    "congestion": ["congestion", "stuffy nose"],
    "chest_pain": ["chest pain", "cardiac issues"],
    "weakness_in_limbs": ["weakness in limbs", "muscle weakness"],
    "fast_heart_rate": ["fast heart rate", "tachycardia"],
    "pain_during_bowel_movements": ["pain during bowel movements", "rectal pain"],
    "pain_in_anal_region": ["pain in anal region", "anal pain"],
    "bloody_stool": ["bloody stool", "hematochezia"],
    "irritation_in_anus": ["irritation in anus", "anal itching"],
    "neck_pain": ["neck pain", "cervical pain"],
    "dizziness": ["dizziness", "lightheadedness"],
    "cramps": ["cramps", "muscle spasms"],
    "bruising": ["bruising", "ecchymosis"],
    "obesity": ["obesity", "overweight"],
    "swollen_legs": ["swollen legs", "edema"],
    "swollen_blood_vessels": ["swollen blood vessels", "vasculitis"],
    "puffy_face_and_eyes": ["puffy face and eyes", "edema"],
    "enlarged_thyroid": ["enlarged thyroid", "goiter"],
    "brittle_nails": ["brittle nails", "nail fragility"],
    "swollen_extremeties": ["swollen extremeties", "edema"],
    "excessive_hunger": ["excessive hunger", "polyphagia"],
    "extra_marital_contacts": ["extra marital contacts", "infidelity"],
    "drying_and_tingling_lips": ["drying and tingling lips", "cheilitis"],
    "slurred_speech": ["slurred speech", "dysarthria"],
    "knee_pain": ["knee pain", "knee joint pain"],
    "hip_joint_pain": ["hip joint pain", "hip arthritis"],
    "muscle_weakness": ["muscle weakness", "muscle atrophy"],
    "stiff_neck": ["stiff neck", "cervical stiffness"],
    "swelling_joints": ["swelling joints", "joint swelling"],
    "movement_stiffness": ["movement stiffness", "muscle rigidity"],
    "spinning_movements": ["spinning movements", "vertigo"],
    "loss_of_balance": ["loss of balance", "ataxia"],
    "unsteadiness": ["unsteadiness", "lightheadedness"],
    "weakness_of_one_body_side": ["weakness of one body side", "hemiparesis"],
    "loss_of_smell": ["loss of smell", "anosmia"],
    "bladder_discomfort": ["bladder discomfort", "urinary issues"],
    "foul_smell_of_urine": ["foul smell of urine", "urinary tract infection"],
    "continuous_feel_of_urine": ["continuous feel of urine", "urinary frequency"],
    "passage_of_gases": ["passage of gases", "flatulence"],
    "internal_itching": ["internal itching", "pruritus"],
    "toxic_look_typhos": ["toxic look typhos", "typhoid fever"],
    "depression": ["depression", "mental health disorders"],
    "irritability": ["irritability", "mood swings"],
    "muscle_pain": ["muscle pain", "myalgia"],
    "altered_sensorium": ["altered sensorium", "confusion"],
    "red_spots_over_body": ["red spots over body", "petechiae"],
    "belly_pain": ["belly pain", "abdominal pain"],
    "abnormal_menstruation": ["abnormal menstruation", "menstrual irregularities"],
    "dischromic_patches": ["dischromic patches", "skin discoloration"],
    "watering_from_eyes": ["watering from eyes", "lacrimation"],
    "increased_appetite": ["increased appetite", "polyphagia"],
    "polyuria": ["polyuria", "frequent urination"],
    "family_history": ["family history", "genetic disorders"],
    "mucoid_sputum": ["mucoid sputum", "respiratory issues"],
    "rusty_sputum": ["rusty sputum", "hemoptysis"],
    "lack_of_concentration": ["lack of concentration", "attention deficit"],
    "visual_disturbances": ["visual disturbances", "blurred vision"],
    "receiving_blood_transfusion": ["receiving blood transfusion", "blood transfusion"],
    "receiving_unsterile_injections": ["receiving unsterile injections", "unsterile injections"],
    "coma": ["coma", "unconsciousness"],
    "stomach_bleeding": ["stomach bleeding", "gastrointestinal bleeding"],
    "distention_of_abdomen": ["distention of abdomen", "abdominal swelling"],
    "history_of_alcohol_consumption": ["history of alcohol consumption", "alcoholism"],
    "fluid_overload.1": ["fluid overload", "edema"],
    "blood_in_sputum": ["blood in sputum", "hemoptysis"],
    "prominent_veins_on_calf": ["prominent veins on calf", "varicose veins"],
    "palpitations": ["palpitations", "heart palpitations"],
    "painful_walking": ["painful walking", "foot pain"],
    "pus_filled_pimples": ["pus filled pimples", "acne"],
    "blackheads": ["blackheads", "comedones"],
    "scurring": ["scurring", "scarring"],
    "skin_peeling": ["skin peeling", "exfoliation"],
    "silver_like_dusting": ["silver like dusting", "skin discoloration"],
    "small_dents_in_nails": ["small dents in nails", "nail abnormalities"],
    "inflammatory_nails": ["inflammatory nails", "nail inflammation"],
    "blister": ["blister", "fluid-filled bumps"],
    "red_sore_around_nose": ["red sore around nose", "nasal irritation"],
    "yellow_crust_ooze": ["yellow crust ooze", "pus-filled crust"]
}

len(keywords_new)

132

In [37]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained model (GPT-2)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Save the GPT-2 model and tokenizer
model.save_pretrained("./gpt2_model")
tokenizer.save_pretrained("./gpt2_tokenizer")

c:\Users\dinhn\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('./gpt2_tokenizer\\tokenizer_config.json',
 './gpt2_tokenizer\\special_tokens_map.json',
 './gpt2_tokenizer\\vocab.json',
 './gpt2_tokenizer\\merges.txt',
 './gpt2_tokenizer\\added_tokens.json')

In [26]:
# Function to match the generated text to symptoms (dummy example)
def match_generated_text_to_symptoms(generated_text):
    # Find matching symptoms in the generated text
    symptoms = {symptom: 0 for symptom in keywords_new.keys()}
    for symptom, keywords in keywords_new.items():
        if any(keyword in generated_text for keyword in keywords):
            symptoms[symptom] = 1
    return symptoms

In [33]:
# Function to use GPT-2 to preprocess the input text for symptom extraction
def map_symptoms_with_llm(text):
    inputs = tokenizer.encode(text, return_tensors="pt")
    outputs = model.generate(inputs, max_length=150, num_return_sequences=1)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Here, you'd implement logic to map the generated text to actual symptoms.
    # This is where you can match it against your symptom_keywords, similar to what you were doing earlier.
    symptoms = match_generated_text_to_symptoms(generated_text)
    return symptoms

# train test split

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [13]:
X = dataset.drop('prognosis', axis=1)
y = dataset['prognosis']

# ecoding prognonsis
le = LabelEncoder()
le.fit(y)
Y = le.transform(y)
    
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=20)

In [14]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(3444, 132)
(1476, 132)
(3444,)
(1476,)


# Training top models

In [15]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score
import numpy as np


# Create a dictionary to store models
models = {
    'SVC': SVC(kernel='linear'),
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'KNeighbors': KNeighborsClassifier(n_neighbors=5),
    'MultinomialNB': MultinomialNB(),
    'LogisticRegression': LogisticRegression(max_iter=1000)
}

# Loop through the models, train, test, and print results
for model_name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)

    # Test the model
    predictions = model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, predictions)
    print(f"{model_name} Accuracy: {accuracy}")

    # Calculate F1 Score
    f1 = f1_score(y_test, predictions, average='weighted')
    print(f"{model_name} F1 Score: {f1}")

    # Calculate Precision
    precision = precision_score(y_test, predictions, average='weighted')
    print(f"{model_name} Precision: {precision}")

    print(classification_report(y_test, predictions))

SVC Accuracy: 1.0
SVC F1 Score: 1.0
SVC Precision: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       1.00      1.00      1.00        43
           2       1.00      1.00      1.00        28
           3       1.00      1.00      1.00        46
           4       1.00      1.00      1.00        42
           5       1.00      1.00      1.00        33
           6       1.00      1.00      1.00        33
           7       1.00      1.00      1.00        39
           8       1.00      1.00      1.00        32
           9       1.00      1.00      1.00        49
          10       1.00      1.00      1.00        37
          11       1.00      1.00      1.00        42
          12       1.00      1.00      1.00        41
          13       1.00      1.00      1.00        32
          14       1.00      1.00      1.00        29
          15       1.00      1.00      1.00        30
          16       1.00   

# single prediction

In [16]:
# selecting logistic regression
clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression()

In [17]:
# predicting probabilities
probabilities = clf.predict_proba(X_test)

In [18]:
# getting top 5 predictions for each sample
top_5_predictions = np.argsort(probabilities, axis=1)[:, -5:]

In [19]:
# printing top 5 predictions for the first sample as an example
print("Top 5 predictions for the first sample:", top_5_predictions[0])

Top 5 predictions for the first sample: [29 22  9 21 40]


In [38]:
import pickle

# Save the logistic regression model
with open('./models/logistic_regression_model.pkl', 'wb') as file:
    pickle.dump(clf, file)

# Load the logistic regression model
with open('./models/logistic_regression_model.pkl', 'rb') as file:
    loaded_clf = pickle.load(file)

# Verify that the loaded model works
print("Loaded model accuracy:", accuracy_score(y_test, loaded_clf.predict(X_test)))

Loaded model accuracy: 1.0


# Recommendation System and Prediction

# load database and use logic for recommendations

In [21]:
sym_des = pd.read_csv("./datasets/symtoms_df.csv")
precautions = pd.read_csv("./datasets/precautions_df.csv")
workout = pd.read_csv("./datasets/workout_df.csv")
description = pd.read_csv("./datasets/description.csv")
medications = pd.read_csv('./datasets/medications.csv')
diets = pd.read_csv("./datasets/diets.csv")

In [34]:
from sklearn.pipeline import Pipeline

#============================================================
# custome and helping functions
#==========================helper funtions================
def helper(diseases):
    all_desc = []
    all_pre = []
    all_med = []
    all_die = []
    all_wrkout = []

    for dis in diseases:
        desc = description[description['Disease'] == dis]['Description']
        desc = " ".join([w for w in desc])
        all_desc.append(desc)

        pre = precautions[precautions['Disease'] == dis][['Precaution_1', 'Precaution_2', 'Precaution_3', 'Precaution_4']]
        pre = [col for col in pre.values]
        all_pre.append(pre)

        med = medications[medications['Disease'] == dis]['Medication']
        med = [med for med in med.values]
        all_med.append(med)

        die = diets[diets['Disease'] == dis]['Diet']
        die = [die for die in die.values]
        all_die.append(die)

        wrkout = workout[workout['disease'] == dis]['workout']
        wrkout = [wrk for wrk in wrkout.values]
        all_wrkout.append(wrkout)

    return all_desc, all_pre, all_med, all_die, all_wrkout

# Define the pipeline
pipeline = Pipeline([
    # ('symptom_mapping', map_symptoms),  # Ensure map_symptoms is defined correctly
    ('symptom_mapping', map_symptoms_with_llm),
    ('create_dataframe', lambda x: pd.DataFrame([x], columns=x.keys())),
    ('disease_classification', loaded_clf)  # Ensure loaded_clf is a trained LogisticRegression model
])

def predict_disease(text):
    symptoms = pipeline['symptom_mapping'](text)
    df = pipeline['create_dataframe'](symptoms)
    probabilities = pipeline['disease_classification'].predict_proba(df)
    top_5_indices = probabilities.argsort()[0][-5:][::-1]
    top_5_diseases = le.inverse_transform(top_5_indices)
    top_5_probabilities = probabilities[0][top_5_indices]
    return list(zip(top_5_diseases, top_5_probabilities))

In [30]:
# Example usage
text = "I have itching and knee pain."
predicted_diseases = predict_disease(text)
print(predicted_diseases)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[('Fungal infection', 0.19663291417364612), ('Drug Reaction', 0.10020550161198315), ('Osteoarthristis', 0.0862648465963258), ('Chronic cholestasis', 0.06039976588455311), ('Allergy', 0.03659220973001586)]


In [31]:
# Test 1
text = input("Enter your symptoms.......")
predicted_diseases = predict_disease(text)

symptoms = pipeline['symptom_mapping'](text)

# Correct the feature names

# Filtering and listing only symptoms with value 1
filtered_symptoms = [symptom for symptom, value in symptoms.items() if value == 1]

# desc, pre, med, die, wrkout = helper(predicted_diseases)

print("=================detected symptoms==================")
i = 1
for item in filtered_symptoms:
    print(i, ": ", item)
    i += 1

print("=================predicted disease============")
for disease, probability in predicted_diseases:
    print(f"{disease}: {probability * 100:.2f}%")

# Get details for the predicted diseases
disease_names = [disease for disease, _ in predicted_diseases]
desc, pre, med, die, wrkout = helper(disease_names)

# Display the details for each predicted disease
# Display the details for each predicted disease
for i, (disease, probability) in enumerate(predicted_diseases):
    print(f"=================Details for {disease} ({probability * 100:.2f}%)==================")
    print("Description:")
    print(desc[i])
    print("Precautions:")
    for j, p in enumerate(pre[i][0]):
        print(f"{j + 1}: {p}")
    print("Medications:")
    for j, m in enumerate(med[i]):
        print(f"{j + 1}: {m}")
    print("Diet:")
    for j, d in enumerate(die[i]):
        print(f"{j + 1}: {d}")
    print("Workout:")
    for j, w in enumerate(wrkout[i]):
        print(f"{j + 1}: {w}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


=================detected symptoms==================
1 :  knee_pain
=================predicted disease============
Osteoarthristis: 12.12%
Allergy: 5.46%
Urinary tract infection: 5.34%
AIDS: 4.91%
Acne: 4.70%
=================Details for Osteoarthristis (12.12%)==================
Description:
Osteoarthristis is a degenerative joint disease that affects the cartilage in joints.
Precautions:
1: acetaminophen
2: consult nearest hospital
3: follow up
4: salt baths
Medications:
1: ['NSAIDs', 'Disease-modifying antirheumatic drugs (DMARDs)', 'Biologics', 'Corticosteroids', 'Joint replacement surgery']
Diet:
1: ['Arthritis Diet', 'Anti-Inflammatory Diet', 'Omega-3-rich foods', 'Fruits and vegetables', 'Whole grains']
Workout:
1: Consume anti-inflammatory foods
2: Include omega-3 fatty acids
3: Consume antioxidant-rich foods
4: Stay hydrated
5: Limit processed foods
6: Include vitamin K-rich foods
7: Consult a healthcare professional
8: Follow medical recommendations
9: Engage in low-impact ex

In [35]:
# Test 2
text = input("Enter your symptoms.......")
predicted_diseases = predict_disease(text)

symptoms = pipeline['symptom_mapping'](text)

# Correct the feature names

# Filtering and listing only symptoms with value 1
filtered_symptoms = [symptom for symptom, value in symptoms.items() if value == 1]

# desc, pre, med, die, wrkout = helper(predicted_diseases)

print("=================detected symptoms==================")
i = 1
for item in filtered_symptoms:
    print(i, ": ", item)
    i += 1

print("=================predicted disease============")
for disease, probability in predicted_diseases:
    print(f"{disease}: {probability * 100:.2f}%")

# Get details for the predicted diseases
disease_names = [disease for disease, _ in predicted_diseases]
desc, pre, med, die, wrkout = helper(disease_names)

# Display the details for each predicted disease
# Display the details for each predicted disease
for i, (disease, probability) in enumerate(predicted_diseases):
    print(f"=================Details for {disease} ({probability * 100:.2f}%)==================")
    print("Description:")
    print(desc[i])
    print("Precautions:")
    for j, p in enumerate(pre[i][0]):
        print(f"{j + 1}: {p}")
    print("Medications:")
    for j, m in enumerate(med[i]):
        print(f"{j + 1}: {m}")
    print("Diet:")
    for j, d in enumerate(die[i]):
        print(f"{j + 1}: {d}")
    print("Workout:")
    for j, w in enumerate(wrkout[i]):
        print(f"{j + 1}: {w}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


=================detected symptoms==================
1 :  shivering
2 :  chills
3 :  headache
4 :  dizziness
=================predicted disease============
Allergy: 47.85%
Hypertension : 18.46%
Paralysis (brain hemorrhage): 4.47%
Malaria: 3.13%
Cervical spondylosis: 3.08%
=================Details for Allergy (47.85%)==================
Description:
Allergy is an immune system reaction to a substance in the environment.
Precautions:
1: apply calamine
2: cover area with bandage
3: nan
4: use ice to compress itching
Medications:
1: ['Antihistamines', 'Decongestants', 'Epinephrine', 'Corticosteroids', 'Immunotherapy']
Diet:
1: ['Elimination Diet', 'Omega-3-rich foods', 'Vitamin C-rich foods', 'Quercetin-rich foods', 'Probiotics']
Workout:
1: Avoid allergenic foods
2: Consume anti-inflammatory foods
3: Include omega-3 fatty acids
4: Stay hydrated
5: Eat foods rich in vitamin C
6: Include quercetin-rich foods
7: Consume local honey
8: Limit processed foods
9: Include ginger in diet
10: Avoid 

In [36]:
# let's use pycharm flask app
# but install this version in pycharm
import sklearn
print(sklearn.__version__)

1.5.0
